# LassoNet instabilities

We demonstrate that the official LassoNet implementation doesn't train well and it consistently obtains almost random performance.

In the official experiments presented in the paper, we grid-searched the $L_1$ penalty hyper-parameter $\lambda \in \{ 0.001, 0.01, 0.1, 1, 10, `auto'\}$ and the hierarchy coefficient $M \in \{ 0.1, 1, 3, 10 \}$. In this toy experiment we use the best hyper-parameter.

In [1]:
import sys
sys.path.insert(0, './src')

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from src.dataset import *
from main import parse_arguments
from lassonet import LassoNetClassifier
from functools import partial

In [3]:
# load dataset
args = parse_arguments([
	'--dataset', 'lung'
])

data_module = create_data_module(args)

Train size: 141

Valid size: 16

Test size: 40

Weights for the classification loss: [0.3525   2.9375   2.35     2.517857]


In [4]:
args.class_weights

array([0.3525  , 2.9375  , 2.35    , 2.517857], dtype=float32)

In [5]:
model = LassoNetClassifier(
	lambda_start = 0.1,
	M = 10,
	n_iters = 200,
	optim = partial(torch.optim.AdamW, lr=1e-4, betas=[0.9, 0.98]),
	hidden_dims=(100, 100, 10),
	class_weight = args.class_weights, # use weighted loss
	dropout=0.2,
	batch_size=8,
	backtrack=True # if True, ensure the objective is decreasing
)

In [6]:
model.path(data_module.X_train, data_module.y_train,
				X_val = data_module.X_valid, y_val = data_module.y_valid);

### Note: LassoNet cannot even fit the training data. It obtains 40% training accuracy on the 'lung' dataset, on which the other models obtain 90% test accuracy, and close to 100% train accuracy.

In [7]:
model.score(data_module.X_train, data_module.y_train) # train balanced accuracy

0.40425531914893614

In [8]:
model.score(data_module.X_valid, data_module.y_valid) # validation balanced accuracy

0.3125